[View in Colaboratory](https://colab.research.google.com/github/Angordil/Analisis-de-registros-de-EEG-mediante-redes-neuronales/blob/master/Vote_model.ipynb)

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip install -q keras

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow
from tensorflow import keras
from tensorflow.python.keras import utils
from tensorflow.python.keras.models import Sequential, save_model, load_model
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.applications import imagenet_utils
from scipy.io import loadmat
import itertools
from sklearn.metrics import confusion_matrix

In [0]:
def doubleconv(X_training, y_training1hot, subj_num):

  num_classes = 6

  model = Sequential()

  model.add(Conv2D(64, (3,3), input_shape = (124,32,1)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D())

  model.add(Conv2D(128,(3,3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D())
  model.add(Dropout(0.3))

  model.add(Flatten())
  model.add(Dense(256))
  model.add(Activation('relu'))
  model.add(Dropout(0.3))
  model.add(Dense(64))
  model.add(Activation('relu'))
  model.add(Dropout(0.3))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

    # Train model
    
  history = model.fit(X_training, y_training1hot, epochs = 24, validation_split = 0.1, shuffle = True, verbose = 1)

  #model.save_weights('drive/TFG/Trial only with subject 1/Weights/weightsVoteModel'+str(subj_num+1)+'.hdf5')

  return model, history

def data_preparation(subj_num):
  # Load data 

  subject = loadmat('drive/TFG/data/S'+str(subj_num+1)+'.mat')

  # Inspect the EEG data for the first trial of the first subject:

  X_training = subject['X_2D']
  
  N = 32
  
  electrodes = 124
  
  # ### Data Preparation

  y_training = subject['categoryLabels'][0]

  # Validation set will be defined when model.fit is done. It'll be a 10% of the training data

  ## 1-Hot Encode Target Output Labels
  num_classes = 6

  y_training1hot = utils.to_categorical(y_training - 1 , num_classes)

  print y_training1hot.shape

  # Reshape training set

  X_training = np.reshape(X_training,(-1, electrodes, N,1)) 
  
  return X_training, y_training1hot, y_training

def conf_matrix(y_validation, X_training, subj_num, model):
  # ### Model Quality Check: Confusion Matrix
  
  y_validation = y_validation[-519:]
  y_validation_predictions = model.predict(X_training[-519:], verbose = 1)

  cnf_matrix = confusion_matrix(y_validation - 1, np.argmax(y_validation_predictions, axis = 1))

  plt.figure()
  class_names = ['Human Body', 'Human Face', 'Animal Body', 'Animal Face', 'Fruit Vegetable', 'Inanimate Object']
  plot_confusion_matrix(cnf_matrix, classes = class_names, title = 'Confusion matrix Subj ' + str(subj_num+1))
  
def plot_confusion_matrix(cm, classes, normalize = False, title = 'Confusion matrix', cmap = plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting 'normalize = True'
    """
    plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 45)
    plt.yticks(tick_marks, classes)

    if normalize:
      cm = cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
      print('Normalized confusion matrix')
    else:
      print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max()/2.
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, cm[i, j], horizontalalignment = 'center', color = 'white' if cm[i, j] > thresh else 'black')

      plt.tight_layout()
      plt.ylabel('True label')
      plt.xlabel('Predicted label')
      

In [0]:
for subj_num in range(0,9):
  X_training, y_training1hot, y_training = data_preparation(subj_num)
  model, history = doubleconv(X_training,y_training1hot, subj_num)
  #conf_matrix(y_training, X_training, subj_num, model)
  save_model(model, 'drive/TFG/Trial only with subject 1/model'+str(subj_num+1)+'.h5py')
  print(str(subj_num) + ' - ok')
  
  import matplotlib.pyplot as plt
  
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  
plt.title('Accuracy over time')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc = 'upper left')
plt.show

In [0]:
### Validation with subject 10

# Load models 
model1 = load_model('drive/TFG/Trial only with subject 1/model1.h5py')

model2 = load_model('drive/TFG/Trial only with subject 1/model2.h5py')

model3 = load_model('drive/TFG/Trial only with subject 1/model3.h5py')

model4 = load_model('drive/TFG/Trial only with subject 1/model4.h5py')

model5 = load_model('drive/TFG/Trial only with subject 1/model5.h5py')

model6 = load_model('drive/TFG/Trial only with subject 1/model6.h5py')

model7 = load_model('drive/TFG/Trial only with subject 1/model7.h5py')

model8 = load_model('drive/TFG/Trial only with subject 1/model8.h5py')

model9 = load_model('drive/TFG/Trial only with subject 1/model9.h5py')

In [0]:
X_validation,y_validation1hot, y_validation = data_preparation(9) 

trial = X_validation[:,:,:,:]

print trial.shape

## Predict class
val1 = np.expand_dims(np.argmax(model1.predict(trial, verbose = 0), axis = 1),axis = 1)
val2 = np.expand_dims(np.argmax(model2.predict(trial, verbose = 0), axis = 1),axis = 1)
val3 = np.expand_dims(np.argmax(model3.predict(trial, verbose = 0), axis = 1),axis = 1)
val4 = np.expand_dims(np.argmax(model4.predict(trial, verbose = 0), axis = 1),axis = 1)
val5 = np.expand_dims(np.argmax(model5.predict(trial, verbose = 0), axis = 1),axis = 1)
val6 = np.expand_dims(np.argmax(model6.predict(trial, verbose = 0), axis = 1),axis = 1)
val7 = np.expand_dims(np.argmax(model7.predict(trial, verbose = 0), axis = 1),axis = 1)
val8 = np.expand_dims(np.argmax(model8.predict(trial, verbose = 0), axis = 1),axis = 1)
val9 = np.expand_dims(np.argmax(model9.predict(trial, verbose = 0), axis = 1),axis = 1)

val = np.concatenate((val1,val2,val3,val4,val5,val6,val7,val8,val9),axis = 1)
print val.shape  

from scipy import stats

predicted_classes = stats.mode(val, axis =1)[0]+1
print predicted_classes.shape

## Compare predicted classes with actual classes
correct = 0
for i in range(0, len(y_validation)):
  if(y_validation[i]==predicted_classes[i]):
    correct = correct +1
  else: correct = correct

accuracy = correct*100/len(y_validation)
print(str(accuracy)+ ' %')